In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trd = pd.read_csv("/kaggle/input/titanic/train.csv")

In [ ]:
ted = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
for data in [trd,ted] :
    mean = trd.Age.mean()
    std =  ted.Age.std()
    is_null= data.Age.isnull().sum()
    randintegers =np.random.randint(mean-std,mean+std,is_null)
    temp_age=data["Age"].copy()
    temp_age[data.Age.isnull()]=randintegers
    data["Age"]=temp_age
tod = trd.copy()
tod = tod.append(ted)

In [ ]:
tod.loc[tod.Embarked.isnull(),'Embarked'] ='S'

In [ ]:
tod.Fare = tod.Fare.fillna(0)
tod.Fare = tod.Fare.astype(int)

In [ ]:

sex_dummies=pd.get_dummies(tod["Sex"])

In [ ]:
Embarked_dummies=pd.get_dummies(tod['Embarked'])
tod=tod.drop(columns=['PassengerId','Sex','Embarked','Ticket'])
tod


In [ ]:
data = [tod]
#titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    #dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    #dataset['Title'] = dataset['Title'].fillna(0)

title_dummies=pd.get_dummies(tod.Title)
title_dummies
cabin_holder=tod['Cabin']
cabin_holder
tod = tod.drop(['Name','Cabin','Title'], axis=1)
tod =pd.concat([tod,sex_dummies,title_dummies,Embarked_dummies],axis=1)


In [ ]:
X_train=tod.iloc[0:891,:]
Y_train=X_train["Survived"]
X_train=X_train.drop(columns='Survived')
X_test=tod.iloc[891:,:]
X_test=X_test.drop(columns='Survived')


In [ ]:
#from sklearn.linear_model import LogisticRegression
#logreg = LogisticRegression()
#logreg.fit(X_train, Y_train)

#Y_pred = logreg.predict(X_test)

#acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
Y_prediction=pd.Series(Y_prediction)
Y_prediction=pd.concat([ted['PassengerId'],Y_prediction],axis=1)
Y_prediction.to_csv('mcsvfile.csv',index=False)
Y_prediction